## More data formats

In [ ]:
import os
from osgeo import ogr
import osgeopy as gp

data_dir = r'osgeopy-data'

In [ ]:
# A function to print out the layers in a data source.
def print_layers(fn):
    ds = ogr.Open(fn, 0)
    if ds is None:
        raise OSError('Could not open {}'.format(fn))
    for i in range(ds.GetLayerCount()):
        lyr = ds.GetLayer(i)
        print('{0}: {1}'.format(i, lyr.GetName()))

In [ ]:
# Try out the function.
print_layers(os.path.join(data_dir, 'Washington', 'large_cities.geojson'))

### Esri file geodatabase

In [ ]:
# Print out layers in an Esri file geodatabase.
fn = os.path.join(data_dir, 'global', 'natural_earth.gdb')
gp.print_layers(fn)

In [ ]:
# Get a layer inside a feature dataset.
ds = ogr.Open(fn)
lyr = ds.GetLayer('countries_10m')

In [ ]:
# Print out some attributes to make sure it worked.
gp.print_attributes(lyr, 5, ['NAME', 'POP_EST'])

In [ ]:
# Export a feature class from geodatabase to a shapefile.
out_folder = os.path.join(data_dir, 'global')
gdb_ds = ogr.Open(fn)
gdb_lyr = gdb_ds.GetLayerByName('countries_110m')
shp_ds = ogr.Open(out_folder, 1)
shp_ds.CopyLayer(gdb_lyr, 'countries_110m')
del shp_ds, gdb_ds

## Web Feature Service

In [ ]:
# Print out layers in a WFS.
url = 'WFS:http://gis.srh.noaa.gov/arcgis/services/watchWarn/MapServer/WFSServer'
gp.print_layers(url)

In [ ]:
# Get the first warning from the WFS. This might take a while because it has
# to download all of the data first.
ds = ogr.Open(url)
lyr = ds.GetLayer(1)
print(lyr.GetFeatureCount())
feat = lyr.GetNextFeature()
print(feat.GetField('prod_type'))

In [ ]:
# Get the first warning from the WFS by limiting the returned features to 1.
ds = ogr.Open(url + '?MAXFEATURES=1')
lyr = ds.GetLayer(1)
print(lyr.GetFeatureCount())

## Attribute filters

In [ ]:
# Get the countries shapefile layer
ds = ogr.Open(os.path.join(data_dir, 'global'))
lyr = ds.GetLayer('ne_50m_admin_0_countries')

In [ ]:
# Apply a filter that finds countries in Asia and see how many records there
# are now.
lyr.SetAttributeFilter('continent = "Asia"')
lyr.GetFeatureCount()

In [ ]:
# You can still get a feature that is not in Asia by using its FID.
lyr.GetFeature(2).GetField('name')

In [ ]:
# Set a new filter that selects South American countries and show the results
# in blue. The old filter is no longer in effect.
lyr.SetAttributeFilter('continent = "South America"')

In [ ]:
# Clear all attribute filters.
lyr.SetAttributeFilter(None)
lyr.GetFeatureCount()

## Spatial filters

In [ ]:
# Get the Germany polygon.
ds = ogr.Open(os.path.join(data_dir, 'global'))
country_lyr = ds.GetLayer('ne_50m_admin_0_countries')

country_lyr.SetAttributeFilter('name = "Germany"')
feat = country_lyr.GetNextFeature()
germany = feat.geometry().Clone()

In [ ]:
city_lyr = ds.GetLayer('ne_50m_populated_places')
city_lyr.GetFeatureCount()
# Use the Germany polygon to set a spatial filter and draw the result as blue
# circles.
city_lyr.SetSpatialFilter(germany)
city_lyr.GetFeatureCount()

In [ ]:
# Add an attribute filter to find the cities with a population over 1,000,000
# and draw them as red squares. Since the spatial filter is still in effect,
# you should only get large cities in Germany.
city_lyr.SetAttributeFilter('pop_min > 1000000')
city_lyr.GetFeatureCount()

In [ ]:
# Remove the spatial filter so now you get global cities with a population
# over 1,000,000. Draw the results as magenta triangles.
city_lyr.SetSpatialFilter(None)
city_lyr.GetFeatureCount()

### To clone or not to clone? 

In [ ]:
# Get a sample layer and the first feature from it.
ds = ogr.Open(os.path.join(data_dir, 'global'))
lyr = ds.GetLayer('ne_50m_admin_0_countries')
feat = lyr.GetNextFeature()

# Now get the feature's geometry and also a clone of that geometry.
geom = feat.geometry()
geom_clone = feat.geometry().Clone()


In [ ]:
# Set the feat variable to a new feature, so the original one is no longer
# accessible.
feat = lyr.GetNextFeature()

# Try to get the area of the cloned polygon. This should work just fine.
print(geom_clone.GetArea())

In [ ]:
# Try to get the area of the original polygon. This should cause Python to
# crash because the polygon is linked to the feature that is no longer
# available.
print(geom.GetArea())